# Ensemble Basics II : Boosting

**실습 전 복습사항**
1. Bagging을 **얼추**안다.
2. Boosting을 **얼추**안다.
3. 사용가능한 ML 모델들이 **다수**있다.
4. ML의 Input과 output 구조를 명확히 이해하고 있다.
5. 뉴럴넷의 아이디어를 이해하고 있다.
6. Cross Validation의 개념을 안다.

**실습 목표**

1. 스태킹!

**실습 데이터**
* Fashion MNIST 
* MNIST (H.W maybe)


Q. 앙상블의 영역에 온다는 것은 어디에 더 무게추가 기울어져 있을까?

'설명과 해석'   &   '예측 정확도'

---------------------------
Rayleigh Kim<br>
biz.rayleigh@gmail.com


## 01. Stacking Example!

**사실상의 모든 절차**<br>
**데이터셋 나누는데는 바리에이션이 엄청 많다**

1. 데이터 셋을 A, B, C 로 나눈다.
    * A, B는 Training Set이라고 생각해도 좋겠다.
    * C는 Validation Set이라고 생각해도 괜찮겠다.
    * Test Set은 저 어딘가에 있다.(이것 까지 고려하면 D로도 나눠야 한다.)
2. 데이터셋 A에서 1계층 모델들을 훈련시킨다.
    * 로지스틱 리그레션, SVM, 디시젼트리 세 분류기를 학습시켰다고 해보자.
    * 그렇다면, 세 종류의 예측값을 뽑을 수 있겠다.
3. B데이터셋을 새로이 재구성한다.
    * 2번의 세 종류의 예측값을 Feature로 한다. Input으로 한다.
    * B데이터셋에 원래 있던 Target을 Y로 한다. Output으로 한다.
4. 재구성한 B데이터셋 위에서 2계층 모델(meta learner)을 학습시킨다.
    * Input이 1계층 모델들의 예측값
    * Output이 B데이터셋의 Y
5. 이제 C데이터 셋 위에서 최종검증이 가능하다.
    * C데이터셋도 재구성 되어야 한다.
 

![모든 설명](https://i1.wp.com/blog.kaggle.com/wp-content/uploads/2017/06/image5.gif?resize=600%2C450)


쉬운 데이터에서 일단 사용법을 익히자!

**이쯤되면 여러분들은 이제 아이리스 전문가**

[쉬운데이터](https://en.wikipedia.org/wiki/Iris_flower_data_set)로 간다!

![sepal과 petal](https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg)

아이리스 데이터!

In [0]:
import time
from sklearn.datasets import load_iris

# 아이리스 데이터셋
iris = load_iris()

In [0]:
iris

In [0]:
# 사용할 것 불러오고!
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import numpy as np

### 1. 데이터셋을 나눈다
(테스트셋 생각 하지말고 나누어보자)

* A, B, C 로 세토막 

In [0]:
X = iris.data
Y = iris.target

A_x, B_x, A_y, B_y = train_test_split(X, Y, test_size = 0.66, shuffle = True)

B_x, C_x, B_y, C_y = train_test_split(B_x, B_y, test_size = 0.5, shuffle = True )

### 2. 1계층 모델들을 학습시킨다.

1. DeisionTreeClassifier 하나
2. Logistic Regression 하나
3. KNeighborClassifier 하나

**A셋 위에서 학습시킨다.**

In [0]:
# 사용할 모델 선언
lev1_DT = DecisionTreeClassifier()
lev1_LR = MLPClassifier(hidden_layer_sizes=(), max_iter=1000)
lev1_KN = KNeighborsClassifier()

In [0]:
# 사용할 모델들 학습

times = []

times.append(time.clock())

lev1_DT.fit(A_x, A_y)

times.append(time.clock())

lev1_LR.fit(A_x, A_y)

times.append(time.clock())

lev1_KN.fit(A_x, A_y)

times.append(time.clock())

for i in range(len(times)-1) :
    print("1계층 {}번째 모델 학습 소요시간 {}".format(i, times[i+1]-times[i]))


### 3. B데이터셋 재구성!

In [0]:
b1_from_1 =  lev1_DT.predict(B_x)
b2_from_1 =  lev1_LR.predict(B_x)
b3_from_1 =  lev1_KN.predict(B_x)

B_x_new = np.column_stack((b1_from_1, b2_from_1, b3_from_1))

### 4. Meta Learner 학습!

* 뉴럴넷을 사용하기로 한다.
* 히든레이어는 다음과 같이 설정한다.
    * n_neurons = B_x_new.shape[-1]
    * hidden_layer_sizes = (n_neurons, n_neurons)

In [0]:
n_neurons = B_x_new.shape[-1]
lev2_NN = MLPClassifier(hidden_layer_sizes = (n_neurons, n_neurons), max_iter=10000,
                       verbose = True)

lev2_NN.fit(B_x_new, B_y)

### 5. C데이터셋 위에서 검증해보자.

In [0]:
# C 데이터셋 재구성

c1_from_1 =  lev1_DT.predict(C_x)
c2_from_1 =  lev1_LR.predict(C_x)
c3_from_1 =  lev1_KN.predict(C_x)

C_x_new = np.column_stack((c1_from_1, c2_from_1, c3_from_1))

#### 1계층 모델들의 성능 평가.

* Precision : 개를 개라고 맞춘 비율
* Recall : 개라고 한 것 중 진짜 개의 비율

In [0]:
from sklearn import metrics

predicted_y = lev1_DT.predict(C_x)

print("Classification report for classifier {} : \n {}\n".format(lev1_DT, metrics.classification_report(C_y, predicted_y)))
print("Classification accuracy : {}".format(metrics.accuracy_score(C_y,predicted_y)))
print("Confusion matrix : \n{}".format(metrics.confusion_matrix(C_y, predicted_y).transpose()))

In [0]:
from sklearn import metrics

predicted_y = lev1_LR.predict(C_x)

print("Classification report for classifier {} : \n {}\n".format(lev1_LR, metrics.classification_report(C_y, predicted_y)))
print("Classification accuracy : {}".format(metrics.accuracy_score(C_y,predicted_y)))
print("Confusion matrix : \n{}".format(metrics.confusion_matrix(C_y, predicted_y).transpose()))

In [0]:
from sklearn import metrics

predicted_y = lev1_KN.predict(C_x)

print("Classification report for classifier {} : \n {}\n".format(lev1_KN, metrics.classification_report(C_y, predicted_y)))
print("Classification accuracy : {}".format(metrics.accuracy_score(C_y,predicted_y)))
print("Confusion matrix : \n{}".format(metrics.confusion_matrix(C_y, predicted_y).transpose()))

#### 2계층 모델의 결과 평가

In [0]:
from sklearn import metrics

predicted_y = lev2_NN.predict(C_x_new)

print("Classification report for classifier {} : \n {}\n".format(lev2_NN, metrics.classification_report(C_y, predicted_y)))
print("Classification accuracy : {}".format(metrics.accuracy_score(C_y,predicted_y)))
print("Confusion matrix : \n{}".format(metrics.confusion_matrix(C_y, predicted_y).transpose()))

## 02. Now Your Turn!


### 위의 01. Stacking Example!을 그대로 따라하자.

* 너무 쉬운 데이터셋 위에, **과도한**작업을 했으니 성능이 무너지는 것은 예사다!
* Discussion
    1. 어떤 모델이 성능이 좋았는가. 
    2. 어떤 모델이 특히 성능이 안좋았는가.


## 3. Stacking More!


### 조금 더 어려운 데이터셋으로 간다!

그리고, 조금 더 복잡한 개념으로 간다!

* K-fold Cross validation을 알아야 좋다.
* 위의 간단한 수준의 Stacking을 이해하지 못했다면, 수업을 듣지 말고, 위의 개념을 복습하는 것이 좋다.

In [0]:
!pip uninstall xgboost
!pip install xgboost
!pip3 install h2o4gpu

In [0]:
import xgboost as xgb

xgb.__version__ ## 0.82 이상이어야 한다.

In [0]:
# 사용할 것 불러오고!
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import time

from mlxtend.classifier import StackingCVClassifier

In [0]:
mnist = tf.keras.datasets.mnist

In [0]:
# mnist데이터 불러오고
(train_x, train_y),(test_x, test_y) = mnist.load_data()

# 스케일링까지 한 번에!
train_x, test_x = train_x / 255.0, test_x / 255.0

# reshape!

train_x = train_x.reshape([train_x.shape[0], -1])
test_x = test_x.reshape([test_x.shape[0], -1])

# shape!

print(train_x.shape)

### 스태킹 계획서

1. 3계층으로 준비할 예정.
    * 1 계층 5모델
    * 2 계층 5모델
    * 3 계층 하나의 모델 ( 서로 다른 방식의 세 모델)

2. 각 계층별로 학습데이터 재구성은 K-fold split을 이용할 것이다.
    * 1계층에서 데이터셋을 5개로 나눈다 A, B, C, D, E
    * 1계층의 5모델은 각각 A, B, C, D, E 중 하나를 제외하고서 학습시킨다.
    * 1계층 모델로 데이터 전체를 재구성한다.
    
    * 2계층에서는, 1계층에서 재구성된 데이터셋을 5개로 나눈다. A, B, C, D, E
    * 2계층의 3모델은 각각 A, B, C, D, E 중 하나를 제외하고서 학습시킨다.
    * 2계층 모델로 데이터를 다시 재구성한다.
    
3. 마지막 계층은 다음과 같이 진행한다.
    * XGBOOST를 사용한다.
    * 모델들의 데이터 구성은 다음과 같이 한다.
        1. 2계층의 예측값들로만 구성된 Feature로 학습
        3. 1계층과 2계층의 모든 결과를 포함한 Feature로 학습

### Level.1

In [0]:
# 사용할 모델 선언

lev1_DT = DecisionTreeClassifier(max_depth=11)
# lev1_KN = KNeighborsClassifier(n_neighbors = 5)  # 너무 오래 걸리는 녀석
lev1_NB = MultinomialNB()
lev1_LR = MLPClassifier(hidden_layer_sizes=(), early_stopping = True, random_state = 2019)
# lev1_SV = SVC() # 매우 오래 걸리는 녀석
# lev1_GB = GradientBoostingClassifier(n_estimators= 10, subsample = 0.9, max_depth = 4, random_state = 2019)
lev1_XG = xgb.XGBClassifier(n_estimators = 30, max_depth = 9,
                                                     **{'gpu_id' : 0,
                                                        'tree_method' : 'gpu_hist'},
                           random_state = 2019)
lev1_RF = RandomForestClassifier(n_estimators = 30, max_depth = 9, random_state = 2019)

lev1_models = [lev1_DT, lev1_NB, lev1_LR, lev1_XG, lev1_RF]


In [0]:
# 사용할 모델들 학습

kf = KFold(n_splits = len(lev1_models) , shuffle = True, random_state = 2019)

i = 0
for train_index, val_index in kf.split(train_x) :
    print("--------------------------------------------------")
    print("1계층, {}번 째 모델 : {} 학습 시작".format(i+1, lev1_models[i]))
    t1 = time.clock()
    lev1_models[i].fit(train_x[train_index], train_y[train_index])
    t2 = time.clock()
    print("{:.3f}secs 소요됨. 학습 완료".format(t2-t1))
    accuracy = metrics.accuracy_score(train_y[val_index], lev1_models[i].predict(train_x[val_index]) )
    print("Validation Accuracy : {:.3f}%".format(accuracy*100))
    i += 1


In [0]:
# New Training Set

t1_from_1 = lev1_DT.predict(train_x)
t2_from_1 = lev1_NB.predict(train_x)
t3_from_1 = lev1_LR.predict(train_x)
t4_from_1 = lev1_XG.predict(train_x)
t5_from_1 = lev1_RF.predict(train_x)

train_x_from_1 = np.column_stack((t1_from_1, t2_from_1, t3_from_1, t4_from_1, t5_from_1  ))

# New Test Set

te1_from_1 = lev1_DT.predict(test_x)
te2_from_1 = lev1_NB.predict(test_x)
te3_from_1 = lev1_LR.predict(test_x)
te4_from_1 = lev1_XG.predict(test_x)
te5_from_1 = lev1_RF.predict(test_x)

test_x_from_1 = np.column_stack((te1_from_1, te2_from_1, te3_from_1, te4_from_1, te5_from_1  ))

### Level.2

In [0]:
# 사용할 모델 선언

lev2_DT = DecisionTreeClassifier(max_depth=8)
lev2_NB = MultinomialNB()
lev2_NN = MLPClassifier(hidden_layer_sizes=(8,8), early_stopping = True, random_state = 2019)
lev2_XG = xgb.XGBClassifier(n_estimators = 20, max_depth = 6,
                                                     **{'gpu_id' : 0,
                                                        'tree_method' : 'gpu_hist'},
                            random_state = 2019)
lev2_RF = RandomForestClassifier(n_estimators = 20, max_depth = 6, random_state = 2019)

lev2_models = [lev2_DT, lev2_NB, lev2_NN, lev2_XG, lev2_RF]


In [0]:
# 사용할 모델들 학습

kf = KFold(n_splits = len(lev2_models) , shuffle = True, random_state = 2020)

i = 0
for train_index, val_index in kf.split(train_x_from_1) :
    print("--------------------------------------------------")
    print("2계층, {}번 째 모델 : {} 학습 시작".format(i+1, lev2_models[i]))
    t1 = time.clock()
    lev2_models[i].fit(train_x_from_1[train_index], train_y[train_index])
    t2 = time.clock()
    print("{:.3f}secs 소요됨. 학습 완료".format(t2-t1))
    accuracy = metrics.accuracy_score(train_y[val_index], lev2_models[i].predict(train_x_from_1[val_index]) )
    print("Validation Accuracy : {:.3f}%".format(accuracy*100))
    i += 1


In [0]:
# New Training Set

t1_from_2 = lev2_DT.predict(train_x_from_1)
t2_from_2 = lev2_NB.predict(train_x_from_1)
t3_from_2 = lev2_NN.predict(train_x_from_1)
t4_from_2 = lev2_XG.predict(train_x_from_1)
t5_from_2 = lev2_RF.predict(train_x_from_1)

train_x_from_2 = np.column_stack((t1_from_2, t2_from_2, t3_from_2, t4_from_2, t5_from_2  ))

# New Test Set

te1_from_2 = lev2_DT.predict(test_x_from_1)
te2_from_2 = lev2_NB.predict(test_x_from_1)
te3_from_2 = lev2_NN.predict(test_x_from_1)
te4_from_2 = lev2_XG.predict(test_x_from_1)
te5_from_2 = lev2_RF.predict(test_x_from_1)

test_x_from_2 = np.column_stack((te1_from_2, te2_from_2, te3_from_2, te4_from_2, te5_from_2  ))



### Level.3 -1

2 계층의 데이터만으로 진행한다.

In [0]:
lev3_XG1 = xgb.XGBClassifier(n_estimators = 20, max_depth = 6,
                            **{'gpu_id' : 0, 'tree_method' : 'gpu_hist'},
                              verbose = True,
                           random_state = 2019)

lev3_XG1.fit(train_x_from_2, train_y)

In [0]:
from sklearn import metrics

predicted_y = lev3_XG1.predict(test_x_from_2)

print("Classification report for classifier {} : \n {}\n".format(lev3_XG1, metrics.classification_report(test_y, predicted_y)))
print("Classification accuracy : {}".format(metrics.accuracy_score(test_y,predicted_y)))
print("Confusion matrix : \n{}".format(metrics.confusion_matrix(test_y, predicted_y).transpose()))

### Level.3 -2

2 계층의 데이터와, 1계층의 3, 4번 컬럼도 가져온다.

In [0]:
train_x_from_12 = np.column_stack((train_x_from_1,train_x_from_2))
test_x_from_12 = np.column_stack((test_x_from_1,test_x_from_2))

In [0]:
lev3_XG2 = xgb.XGBClassifier(n_estimators = 40, max_depth = 8,
                            **{'gpu_id' : 0, 'tree_method' : 'gpu_hist'},
                              verbose = True,
                           random_state = 2019)

lev3_XG2.fit(train_x_from_12, train_y)

In [0]:
from sklearn import metrics

predicted_y = lev3_XG2.predict(test_x_from_12)

print("Classification report for classifier {} : \n {}\n".format(lev3_XG2, metrics.classification_report(test_y, predicted_y)))
print("Classification accuracy : {}".format(metrics.accuracy_score(test_y,predicted_y)))
print("Confusion matrix : \n{}".format(metrics.confusion_matrix(test_y, predicted_y).transpose()))

## 04. Now Your Turn Again!


### 위의 03. Stacking More!를 그대로 따라하자.


## 5. Competition on Fashion MNIST

아무 제한 없다. 배운 것 전부 사용하여 성능을 올려라!

In [0]:
import tensorflow as tf
import random as rd
import matplotlib.pyplot as plt

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.fashion_mnist.load_data()

labels = {0 : 'T-shirt/top',  
1 : 'Trouser'  ,
2 : 'Pullover'  ,
3 : 'Dress' , 
4 : 'Coat'  ,
5 : 'Sandal'  ,
6 : 'Shirt'  ,
7 : 'Sneaker'  ,
8 : 'Bag'  ,
9 : 'Ankle boot'}

In [0]:
'''
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print('id = {}'.format(id))
print('다음 그림은 {} 입니다.'.format(labels[test_y[id]]))
plt.imshow(test_x[id])
plt.show()

In [0]:
# reshape!
train_x_flatten = train_x.reshape([train_x.shape[0],-1])
test_x_flatten = test_x.reshape([test_x.shape[0],-1])

print(train_x_flatten.shape, train_y.shape)

In [0]:
## Your Code